In [ ]:
import pandas as pd
from langchain import PromptTemplate
from langchain.chains.question_answering import load_qa_chain
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma


In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI


In [ ]:
from dotenv import load_dotenv
load_dotenv()
import google.generativeai as genai
import os
genai.configure(api_key=os.environ.get("GOOGLE_API_KEY"))
model = ChatGoogleGenerativeAI(model="gemini-pro",
                             temperature=0.3)


In [ ]:
#using the langchain for accesin gthe gemmini api
from langchain_google_genai import ChatGoogleGenerativeAI


In [ ]:
llm = ChatGoogleGenerativeAI(model="gemini-pro")
result = llm.invoke("What is Mixture of Experts?")


In [ ]:
result
result.content

In [ ]:
#now chattting with documents
# !pip install pypdf
#extract text from pdf
pdf_path = "Basics of Diabetes Management.pdf"
pdf_loader = PyPDFLoader(pdf_path)
pages = pdf_loader.load_and_split()
#print(pages[3].page_content)


In [ ]:
pages
print(pages[0].page_content)


In [ ]:

context = "\n".join(str(p.page_content) for p in pages[:30])
print("The total words in the context: ", len(context))

In [ ]:
#prompt desing
prompt_template = """you are diabetes Instructor So , Answer the question as precise as possible using the provided context. If the answer is
                    not contained in the context, say "answer not available in context" \n\n
                    Context: \n {context}?\n
                    Question: \n {question} \n
                    Answer:
                  """

In [ ]:
prompt = PromptTemplate(
    template=prompt_template, input_variables=["context", "question"]
)
stuff_chain = load_qa_chain(model, chain_type="stuff", prompt=prompt)


In [ ]:
question = "What is sugar levels?"


stuff_answer = stuff_chain(
    {"input_documents": pages[0:1], "question": question}, return_only_outputs=True
)

print(stuff_answer)


In [ ]:
question = "What is insulin terapy?"


stuff_answer = stuff_chain(
    {"input_documents": pages[0:1], "question": question}, return_only_outputs=True
)

print(stuff_answer['output_text'])


In [ ]:
question = "who am i?"


stuff_answer = stuff_chain(
    {"input_documents": pages[0:1], "question": question}, return_only_outputs=True
)

print(stuff_answer['output_text'])


In [1]:
.
from langchain_google_genai import ChatGoogleGenerativeAI

from dotenv import load_dotenv
load_dotenv()
import google.generativeai as genai
import os
genai.configure(api_key=os.environ.get("GOOGLE_API_KEY"))
model = ChatGoogleGenerativeAI(model="gemini-pro",
                             temperature=0.3)

#using the langchain for accesin gthe gemmini api
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(model="gemini-pro")
result = llm.invoke("What is Mixture of Experts?")

result
result.content
#now chattting with documents
# !pip install pypdf
#extract text from pdf
pdf_path = "Basics of Diabetes Management.pdf"
pdf_loader = PyPDFLoader(pdf_path)
pages = pdf_loader.load_and_split()
#print(pages[3].page_content)

pages
print(pages[0].page_content)

context = "\n".join(str(p.page_content) for p in pages[:30])
print("The total words in the context: ", len(context))
#prompt desing
prompt_template = """you are diabetes Instructor So , Answer the question as precise as possible using the provided context. If the answer is
                    not contained in the context, say "answer not available in context" \n\n
                    Context: \n {context}?\n
                    Question: \n {question} \n
                    Answer:
                  """

prompt = PromptTemplate(
    template=prompt_template, input_variables=["context", "question"]
)
stuff_chain = load_qa_chain(model, chain_type="stuff", prompt=prompt)

question = "What is sugar levels?"


stuff_answer = stuff_chain(
    {"input_documents": pages[0:1], "question": question}, return_only_outputs=True
)

print(stuff_answer)

question = "What is insulin terapy?"


stuff_answer = stuff_chain(
    {"input_documents": pages[0:1], "question": question}, return_only_outputs=True
)

print(stuff_answer['output_text'])

question = "give Dietary Recommendations?"


stuff_answer = stuff_chain(
    {"input_documents": pages[0:1], "question": question}, return_only_outputs=True
)

print(stuff_answer['output_text'])

question = "who am i?"


stuff_answer = stuff_chain(
    {"input_documents": pages[0:1], "question": question}, return_only_outputs=True
)

print(stuff_answer['output_text'])

#now RAGpipeline : Embedding + LLM 
# additoinal our wish

Understanding
Diabetes
●
Definition
and
Types
:
Diabetes
is
a
chronic
condition
characterized
by
high
levels
of
glucose
in
the
blood.
There
are
two
main
types:
Type
1,
which
is
typically
diagnosed
in
children
and
young
adults
and
involves
the
body
not
producing
insulin,
and
Type
2,
which
is
more
common
and
involves
insulin
resistance
or
insufficient
insulin
production.
●
Symptoms
:
Common
symptoms
include
frequent
urination,
increased
thirst,
extreme
fatigue,
blurred
vision,
and
slow-healing
cuts
or
bruises.
Treatment
Methods
●
Insulin
Therapy
:
Essential
for
individuals
with
Type
1
diabetes
and
sometimes
necessary
for
Type
2
diabetes.
Insulin
must
be
injected
or
used
with
an
insulin
pump.
●
Oral
Medications
:
Commonly
prescribed
for
Type
2
diabetes
to
help
manage
blood
glucose
levels.
Examples
include
Metformin,
which
can
reduce
glucose
production
in
the
liver.
●
Lifestyle
Changes
:
Includes
managing
diet,
regular
physical
activity,
and
maintaining
a
healthy
weight.
Dietary
Recommenda

c:\Users\poppo\anaconda3\envs\dlproj\Lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


{'output_text': 'Answer not available in context'}
Insulin Therapy is essential for individuals with Type 1 diabetes and sometimes necessary for Type 2 diabetes. Insulin must be injected or used with an insulin pump.
Dietary Recommendations:
● Carbohydrate Management: It's crucial for diabetics to monitor carbohydrate intake to manage blood sugar levels. Incorporating complex carbohydrates like whole grains, fruits, and vegetables is recommended.
● Meal Planning: Eating regular meals, balancing macronutrients, and portion control are important to stabilize blood sugar levels.
● Avoiding Sugary Foods: Limiting foods high in sugar and fat to prevent spikes in blood sugar levels.
Diabetes Instructor
